In [1]:
import pandas as pd
import numpy as np

# load all csv

crm_df = pd.read_csv(r"C:\Users\harsh\OneDrive\Desktop\data analysis\Revenue_leakage and KPI analytics project\data\raw\crm1.csv")
device_df = pd.read_csv(r"C:\Users\harsh\OneDrive\Desktop\data analysis\Revenue_leakage and KPI analytics project\data\raw\device1.csv")
rev_df = pd.read_csv(r"C:\Users\harsh\OneDrive\Desktop\data analysis\Revenue_leakage and KPI analytics project\data\raw\rev1.csv")
assurance_df = pd.read_csv(r"C:\Users\harsh\OneDrive\Desktop\data analysis\Revenue_leakage and KPI analytics project\data\raw\telecom_revenue_assurance_dataset.csv")

In [9]:
def clean_summary(df, name):
    print(f"\n Summary for: {name}")
    print("Shape:", df.shape)
    print("Missing Values:\n", df.isnull().sum())
    print("Duplicates:", df.duplicated().sum())
    print("Data Types:\n", df.dtypes)



In [10]:
clean_summary(crm_df,"CRM")
clean_summary(device_df,"Device")
clean_summary(rev_df,"Revenue")
clean_summary(assurance_df,"Assurance")



 Summary for: CRM
Shape: (13627493, 6)
Missing Values:
 msisdn                 0
gender           2679804
year_of_birth       2940
system_status          0
mobile_type            0
value_segment          0
dtype: int64
Duplicates: 4373
Data Types:
 msisdn            object
gender            object
year_of_birth    float64
system_status     object
mobile_type       object
value_segment     object
dtype: object

 Summary for: Device
Shape: (2440100, 6)
Missing Values:
 msisdn             0
imei_tac           0
brand_name     30446
model_name     30446
os_name       677714
os_vendor     660472
dtype: int64
Duplicates: 0
Data Types:
 msisdn        object
imei_tac      object
brand_name    object
model_name    object
os_name       object
os_vendor     object
dtype: object

 Summary for: Revenue
Shape: (1704601, 3)
Missing Values:
 msisdn         0
week_number    0
revenue_usd    0
dtype: int64
Duplicates: 0
Data Types:
 msisdn          object
week_number      int64
revenue_usd    float64
d

In [11]:
# drop duplicates
crm_df.drop_duplicates(inplace=True)

# drops rows with null year_of_birth cause there are very few rows
crm_df = crm_df[crm_df['year_of_birth'].notna()]

# Convert year_of_birth to Int64
crm_df['year_of_birth'] = crm_df['year_of_birth'].astype('Int64')

# fill missing genders as 'unknown'
crm_df['gender']= crm_df['gender'].fillna('Unknown')

# standardize column names 
crm_df.columns = crm_df.columns.str.lower()

In [14]:
print(crm_df.shape)

(13620180, 6)


In [ ]:
# drop rows where both brand and model are missing
device_df = device_df[~(device_df['brand_name'].isnull() & device_df['model_name'].isnull())]

# fill missing os and vendor as "unknown"
device_df['os_name'] = device_df['os_name'].fillna("unknown")
device_df['os_vendor'] = device_df['os_vendor'].fillna("unknown")

# standardize column names

device_df.columns = device_df.columns.str.lower()


In [16]:
print(device_df.shape)

(2409654, 6)


In [17]:
# Remove any negative revenue if exists
rev_df = rev_df[rev_df['revenue_usd']>=0]

# standardize column names
rev_df.columns =rev_df.columns.str.lower()

In [18]:
# standardize column names
assurance_df.columns = assurance_df.columns.str.lower()

# Rename CustomerID to msisdn if it maps to the same user
assurance_df.rename(columns={'customerid':'msisdn'},inplace=True)

In [19]:
crm_df.reset_index(drop=True, inplace=True)
device_df.reset_index(drop=True, inplace=True)
rev_df.reset_index(drop=True, inplace=True)
assurance_df.reset_index(drop=True, inplace=True)


In [20]:
# merging CRM, Device and Revenue data

crm_device_df = pd.merge(crm_df,device_df, on='msisdn',how="left")

# merge with revenue
merged_df = pd.merge(crm_device_df,rev_df,on='msisdn',how='left')

# final shape
print("shape of merged_df :", merged_df.shape)


shape of merged_df : (15044939, 13)


In [21]:
print(merged_df.head())

                             msisdn  gender  year_of_birth system_status  \
0  aeef4233d9ad34e41f7ecf48d64646f8    MALE           1985        ACTIVE   
1  2397d009e705bc8a2654cbb3f480cc59    MALE           1958        ACTIVE   
2  6f05d738919f9283322bae17dc366bf5    MALE           1976        ACTIVE   
3  45352d9d126f86f40c7eee79a82c833c    MALE           1996        ACTIVE   
4  959b00a279e2785cfa81728338c324f7  FEMALE           1960        ACTIVE   

  mobile_type value_segment imei_tac brand_name model_name os_name os_vendor  \
0     Prepaid        Tier_3      NaN        NaN        NaN     NaN       NaN   
1     Prepaid        Tier_3      NaN        NaN        NaN     NaN       NaN   
2     Prepaid        Tier_3      NaN        NaN        NaN     NaN       NaN   
3     Prepaid        Tier_3      NaN        NaN        NaN     NaN       NaN   
4     Prepaid        Tier_3      NaN        NaN        NaN     NaN       NaN   

   week_number  revenue_usd  
0          NaN          NaN  
1 

In [24]:
merged_df.to_csv(r'C:\Users\harsh\OneDrive\Desktop\data analysis\Revenue_leakage and KPI analytics project\data\cleaned\merged_data.csv',index=False)


In [25]:
display(merged_df.isnull().sum())

msisdn                  0
gender                  0
year_of_birth           0
system_status           0
mobile_type             0
value_segment           0
imei_tac         12914205
brand_name       12914205
model_name       12914205
os_name          12914205
os_vendor        12914205
week_number      13543945
revenue_usd      13543945
dtype: int64

In [27]:
display(crm_df.isnull().sum())
display(rev_df.isnull().sum())
display(device_df.isnull().sum())

msisdn           0
gender           0
year_of_birth    0
system_status    0
mobile_type      0
value_segment    0
dtype: int64

msisdn         0
week_number    0
revenue_usd    0
dtype: int64

msisdn        0
imei_tac      0
brand_name    0
model_name    0
os_name       0
os_vendor     0
dtype: int64

In [28]:
crm_df.to_csv(r"C:\Users\harsh\OneDrive\Desktop\data analysis\Revenue_leakage and KPI analytics project\data\cleaned\customer_df.csv")
rev_df.to_csv(r"C:\Users\harsh\OneDrive\Desktop\data analysis\Revenue_leakage and KPI analytics project\data\cleaned\Revenue_data.csv")
device_df.to_csv(r"C:\Users\harsh\OneDrive\Desktop\data analysis\Revenue_leakage and KPI analytics project\data\cleaned\device_data.csv")